In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <u>Libraries</u>

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from PIL import Image
import cv2
import glob

import matplotlib.pyplot as plt
import os
import json
from tqdm.notebook import tqdm
import random


from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, 
    ShiftScaleRotate, CenterCrop, Resize, RandomGamma, RandomShadow, RandomGridShuffle
)

from albumentations.pytorch import ToTensorV2

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

# <u> Helper Function </u>

# <u> Display the image example </u>

In [ ]:
# Create the path for the image in the test image folder
t_path = '../input/cassava-leaf-disease-classification/test_images'
test_image_path = os.path.join(t_path, '2216849948.jpg')

# Create PIL image
im=Image.open(test_image_path)

# From PIL image to tensor
tensor_im = transforms.ToTensor()(im)

# From tensor to PIL image
tensor_to_pil = transforms.ToPILImage()(tensor_im)
print("Image Size :", tensor_to_pil.size)
print("Image channels : " ,tensor_to_pil.mode)

image = cv2.imread(test_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)



# <u>Load the train csv file</u>

In [ ]:
test_image_path = '../input/cassava-leaf-disease-classification/test_images/'

train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"
train_df = pd.read_csv(train_csv_path)

train_df.head()

## <u> Number occurence of each label </u>

As we can see, the label 3 is much more present than the others

In [ ]:
train_df['label'].value_counts()

## <u>Info of the df</u>
We have 21397 images and the type of the labels is int64

In [ ]:
train_df.info()

## Meanings of the labels

In [ ]:
label_file = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"

labels = json.load(open(label_file))
labels

# Plan

 - Create the Dataset object : 
     - First separate the file names and their labels
     - __init__() For each of the image id in the dataframe : 
     
         - create the filename from the 'image_id' column
         - load it as an image
         - convert it pil_img = Image.open(img_path) -> The To_tensor transform will transform it into an image for us
         - Get its label from the 'label' column
         
     - Create the __len__() method which will be the number of training examples
     - Create the __getitem__() method 
 - Create the Transform object TO DO
 - Create the DataLoader   
 - Create the CNN object
 - Create loss and optim objects
 - Make the training loop

# <u> Dataset </u>

I don't know if it is normal but it takes ages to construct.

It was because I was loading all the images in the constructor.




class Data(Dataset):
    def __init__(self, Dataframe, path, Transform):
        
        #This function returns the images and their labels
        def getimages(Dataframe, path):
            images = list()
            labels = list()
            for i in range(len(Dataframe)):
                print(i/len(Dataframe))
                name,label = list(zip(train_df.image_id, train_df.label))[i]
                
                img_path   = os.path.join(path, name)
                pil_image  =  Image.open(img_path)
                
                images.append(pil_image)
                labels.append(label)
                
            return images, labels
    
        self.images, self.labels = getimages(Dataframe, path)
        
        self.transform = Transform
        
        # In case I forget the images dimensions
        self.images_dim = self.images[0].size
      
    def __len__(self): return len(self.images)
    
    def __getitem__(self, index):
        
        image = self.images[index]
        label = self.labels[index]
        
        return self.transform(image), label

In [ ]:
names_train = train_df['image_id'].values
labels_train = train_df['label'].values

names_test = [name for name in (os.listdir(test_image_path))]
names_test

In [ ]:
# inspired by https://www.kaggle.com/ateplyuk/simplest-starting-code-cassava-leaf-pytorch

class Data(Dataset):
    
    def __init__(self, directory, names, labels, Transform):
        self.dir = directory
        self.names = names
        self.labels = labels
        self.transform = Transform
        
    def __len__(self): return len(self.names)
    
    def __getitem__(self, index):
        
        
        #image = cv2.imread(os.path.join(self.dir, self.names[index]))
        #image = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2RGB)
        image = Image.open(os.path.join(self.dir, self.names[index]))       
        label = self.labels[index]
        
        if self.dir == '../input/cassava-leaf-disease-classification/train_images':
            return self.transform(image), label
        
        elif self.dir == '../input/cassava-leaf-disease-classification/test_images/':
            img_name = self.names[index]

            return index, self.transform(image), img_name
            
        else: 
            print('incorrect directory')
    

In [ ]:
names_train = train_df['image_id'].values
labels_train = train_df['label'].values

proto_names = train_df['image_id'][:5000].values
proto_labels = train_df['label'][:5000].values

names_test = [name for name in (os.listdir(test_image_path))]

training_path = '../input/cassava-leaf-disease-classification/train_images'
test_image_path = '../input/cassava-leaf-disease-classification/test_images/'


IM_SIZE = 256

p = 0.2

# Test with albumentation
Album = Compose([
            CenterCrop(int(IM_SIZE/2), int(IM_SIZE/2), p = p),
            Resize(IM_SIZE, IM_SIZE),
            Transpose(p = p),
            HorizontalFlip(p = p),
            VerticalFlip(p),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=p),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=p),
            RandomGamma(gamma_limit=(50, 250), p=p), 
            
            GaussNoise(var_limit=(2500, 3000), mean=0, p = p),
            IAAAdditiveGaussianNoise(loc=70, scale=(2.5500000000000003, 12.75), p = p),
            RandomGridShuffle(grid=(2, 2), p = p),
            OpticalDistortion(distort_limit=0.75, shift_limit=0.75,p = p),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),], p=1.)


# Test with torchvision transform api
Transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomRotation(180),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


#Train_dataset = Data(directory = training_path, names = proto_names, labels = proto_labels, Transform = Transform)
#Test_dataset  = Data(directory = test_image_path, names = names_test, labels = None, Transform = Album)

In [ ]:
proto_names.shape

# <u>DataLoader</u>

In [ ]:
#batchsize = 16

#train_dl = DataLoader(dataset = Train_dataset, batch_size = batchsize)

# <u>CNN</u>

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3,12,3,1)
        self.conv2 = nn.Conv2d(32,24,3,1)
        self.fc1 = nn.Linear(1016064, 512)
        self.fc2 = nn.Linear(512, 5)
        
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = torch.flatten(nn.functional.max_pool2d(nn.functional.relu(self.conv2(x)),2),1)
        print(x.shape)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output
    



# Loss and Optimizer

In [ ]:
#optimizer = torch.optim.Adam(params = model.parameters() , lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#criterion = torch.nn.CrossEntropyLoss()


# Training Loop

I forgot to divide the training accuracy by the batchsize inside each epoch with resulted in accuracy above 1

In [ ]:
def train(model, epochs, train_dl, valid_dl, optimizer, scheduler, criterion, device, batchsize):
    print('='*20)
    print('Starting Training')
    print('='*20)
    Loss_tracker = []
    Train_Accuracy_tracker = []
    Valid_Accuracy_tracker = []
    
    for epoch in range(epochs):
        
        print('Starting epoch', epoch+1)
        model.train()
        Training_loss = 0
        Training_accuracy = 0
        Cost = 0
        
        
        Path = 'CassavaAtEpch' + str(epoch)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'cost': Cost}, Path)
        
        
        i = 0
        for (image, label) in tqdm(train_dl, desc="Training"):
            i+=1                         
            image = image.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            logit = model(image)
            loss = criterion(logit, label)
            loss.backward()
            optimizer.step()
           
            prediction = torch.argmax(logit,1)
            
            Training_accuracy += sum(prediction == label)/batchsize
            Training_loss += loss.detach().item()
            
        Cost = Training_loss/i
        print(i)
        print('Epoch : ', epoch+1, 'Loss : ', Cost)
        print('Epoch : ', epoch+1, 'Accuracy : ', Training_accuracy/i)
        
        scheduler.step(Cost)
        
        
        Valid_accuracy = 0
        model.eval()
        i = 0
        for (image, label) in tqdm(valid_dl, desc = 'Evaluating'):
            i+=1
            image = image.to(device)
            label = label.to(device)
            
            logit = model(image)
           
            prediction = torch.argmax(logit,1)
            
            Valid_accuracy += sum(prediction == label)/batchsize            
        
        print('Epoch : ', epoch+1, 'Valid Accuracy : ', Valid_accuracy/i) 
        
        Loss_tracker.append(Cost)
        Train_Accuracy_tracker.append(Training_accuracy/i)
        Valid_Accuracy_tracker.append(Valid_accuracy/i)
        
        Path = 'CassavaAtEpch' + str(epoch)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'cost': Cost}, Path)
        
        
        
        

# Training

I first made the mistake of training on the whole dataset which was bad for validation and for evaluation. I decided to use only 5000 images at first to see what would improve the accuracy in less time. Surprisingly, 5000 and 23000 images provided the same performences of 61% accuracy. Then I used the randomsplit function of Pytorch to get a proper validation set.

After looking at what other people were saying, the first big problem was the data imbalance. Which I kind of solved by adding weights to my loss function. Now after 10 epochs I have 71% accuracy on the train set and 68% on the validation set and I am happy because it doesn't seem to be overfitting yet and the accuracy can still climb.

After going to sleep I reached 80% of accuracy after 30 epochs but unfortunately The process stopped because my computer shutdowned.

Then I tried to add (a lot of) data augmentation with albumentation since everybody is using it but the performence went back to 60%. Maybe I was adding to much, maybe I did not write it the right way. I need to do further research. My get is that my resnet18 model was not deep enough to deal with such variety of augmentation. I should try with a more complex network. Actually, when I went back to my basic augmentation (but kept openning the images with cv2) the performence was very random. There might be something with cv2 I don't get yet.

#dataset = Data(directory = training_path, names = proto_names, labels = proto_labels, Transform = Transform)

dataset = Data(directory = training_path, names = names_train, labels = labels_train, Transform = Transform)

train_dataset, valid_dataset = torch.utils.data.random_split(dataset = dataset, lengths = [18000, 3397])

batchsize = 12
train_dl = DataLoader(dataset = train_dataset, batch_size = batchsize, shuffle = True, num_workers = 4)
valid_dl = DataLoader(dataset = valid_dataset, batch_size = batchsize, shuffle = True, num_workers = 3)

#model = CNN()

#model = torchvision.models.resnet18(pretrained = False)
#model.fc = nn.Linear(512, 5, bias=True)

model = timm.create_model('tf_efficientnet_b5_ns', pretrained = False)
model.classifier = nn.Linear(model.classifier.in_features, 5)


optimizer = torch.optim.Adam(params = model.parameters() , lr=0.01, 
                             betas=(0.9, 0.999), eps=1e-08, 
                             weight_decay=0, amsgrad=False)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.2, patience = 10, verbose = True)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weight = torch.tensor([1,1,1,0.3,1], device = DEVICE)
criterion = torch.nn.CrossEntropyLoss(weight = weight)


model = model.to(DEVICE)

PATH = '../input/weights/CassavaAtEpch2 (1)'

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.2, patience = 10, verbose = True)

train(model = model, epochs = 30, train_dl = train_dl, 
      valid_dl = valid_dl, optimizer = optimizer ,
      scheduler = scheduler, criterion = criterion, device = DEVICE, batchsize = batchsize)

#train(model = model, epochs = 3, train_dl = train_dl, 
      valid_dl = valid_dl, optimizer = optimizer ,
      scheduler = scheduler, criterion = criterion, device = DEVICE, batchsize = batchsize)

In [ ]:
0.7007415491700173

In [ ]:
model = timm.create_model('tf_efficientnet_b5_ns', pretrained = False)
model.classifier = nn.Linear(model.classifier.in_features, 5)
PATH = '../input/cassava-last-weights/CassavaAtEpch29'
optimizer = torch.optim.Adam(params = model.parameters() , lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_dataset = Data(directory = test_image_path, names = names_test, labels = labels_train, Transform = Transform)

test_dataloader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=1,
        num_workers=0,
        shuffle=False,
        pin_memory=False,
    )

for i, img ,img_name in test_dataloader:
    break

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

submission = pd.DataFrame(columns=('image_id','label'))
for i, img ,img_name in test_dataloader:
        model.eval()
        img = img.to(device)
        index = i.item()
        label = torch.argmax(model(img)).item()
        submission = submission.append([{'image_id':img_name[0],'label':label}],ignore_index=True)
submission.to_csv('submission.csv',index=False)

In [ ]:
submission